# SCRAPE THE GRADE DATA OF THE SERIE A PLAYERS FROM FANTACALCIO.IT

In [ ]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import unicodedata
import pandas as pd
import time
import re

<br>

SETING UP FUNCTIONS
<br>
<br>

In [ ]:
# funtion to close the opened pop-up
def pop_up_closer():
    try:
        driver.find_element(By.CLASS_NAME, 'qc-cmp2-close-icon').click()
        time.sleep(20)
    except:
        pass

    try:
        driver.find_element(By.ID, 'pushengage-subscription-reminder-prompt-button').click()
    except:
        pass

    try:
        driver.find_element(By.ID, 'pushengage-opt-in-6-close').click()
    except:
        pass

In [ ]:
# function to log us into the website
def log_in(user_id, password):
    # log-in logo button
    driver.find_element(By.CLASS_NAME, 'btn-primary').click()
    # log-in button
    driver.find_element(By.CLASS_NAME, 'off-user-logged').click()
    # pop up closing
    pop_up_closer()

    # credentials insertion:
    # username field
    username_field = driver.find_element(By.NAME, "username")
    username_field.send_keys(user_id)
    # password fiekd
    password_field = driver.find_element(By.NAME, "password")
    password_field.send_keys(password)
    # enter button
    driver.find_element(By.CLASS_NAME, 'btn-block').click()

In [ ]:
# funtion to obtain data we need
def scrape_data(season_list=[]):

    df = pd.DataFrame()

    for season in season_list:
        # season dropdown button
        season_menu = driver.find_element(By.ID, 'season')
        season_menu.click()

        # season selection
        season_select = Select(season_menu)
        season_select.select_by_value(season)

        for i in range(1, 39):
            # matchday selection
            matchweek = driver.find_element(By.ID, 'matchweek')
            matchday_select = Select(matchweek)
            matchday_select.select_by_value(f"{i}")

            # data tables
            tables = driver.find_elements(By.CLASS_NAME, 'grades-table')

            for table in tables:
                # extract the team
                thead = table.find_element(By.TAG_NAME, 'thead')
                team = thead.find_element(By.CLASS_NAME, 'team-name').text.strip()

                # all rows in the table body
                tbody = table.find_element(By.TAG_NAME, 'tbody')
                rows = tbody.find_elements(By.TAG_NAME, 'tr')

                for row in rows:
                    # players_data
                    descriptive_data = row.find_elements(By.TAG_NAME, 'td')[0]
                    grades_data = row.find_elements(By.TAG_NAME, 'td')[1]
                    bonus_data = row.find_elements(By.TAG_NAME, 'td')[2]

                    # player name and role
                    name = descriptive_data.find_element(By.CLASS_NAME, 'player-name').text
                    role_element = descriptive_data.find_element(By.CLASS_NAME, 'role')
                    role = role_element.get_attribute('data-value')
                    if role == 'all':
                        continue

                    # player grades
                    grades_block = grades_data.find_elements(By.CLASS_NAME, 'pill')[0]

                    grade_element = grades_block.find_element(By.CLASS_NAME, 'player-grade ')
                    grade = grade_element.get_attribute('data-value')
                    if grade == '6*':
                        continue
                    elif ',' in grade:
                        grade = grade.replace(',', '.')
                        grade = float(grade)
                    else:
                        grade = float(grade)

                    fanta_grade_element = grades_block.find_element(By.CLASS_NAME, 'player-fanta-grade')
                    fanta_grade = fanta_grade_element.get_attribute('data-value')
                    if fanta_grade == '6*':
                        continue
                    elif ',' in fanta_grade:
                        fanta_grade = fanta_grade.replace(',', '.')
                        fanta_grade = float(fanta_grade)
                    else:
                        fanta_grade = float(fanta_grade)

                    # player bonus
                    goals_scored_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[0]
                    goals_scored = int(goals_scored_element.get_attribute('data-value'))

                    goals_conceded_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[1]
                    goals_conceded = int(goals_conceded_element.get_attribute('data-value'))

                    autogoals_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[2]
                    autogoals = int(autogoals_element.get_attribute('data-value'))

                    penalties_scored_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[3]
                    penalties_scored = int(penalties_scored_element.get_attribute('data-value'))

                    penalties_missed_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[4]
                    penalties_missed = int(penalties_missed_element.get_attribute('data-value'))

                    penalties_saved_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[5]
                    penalties_saved = int(penalties_saved_element.get_attribute('data-value'))

                    assists_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[6]
                    assists = int(assists_element.get_attribute('data-value'))

                    player_of_the_match_element = bonus_data.find_elements(By.CLASS_NAME, 'player-bonus')[7]
                    player_of_the_match = int(player_of_the_match_element.get_attribute('data-value'))

                    # update the dataframe
                    new_row = pd.DataFrame({
                        'Name': [name],
                        'Team': [team],
                        'Role': [role],
                        'Grade': [grade],
                        'FantaGrade': [fanta_grade],
                        'Goals Scored': [goals_scored],
                        'Goals Conceded': [goals_conceded],
                        'Autogoals': [autogoals],
                        'Penalties Scored': [penalties_scored],
                        'Penalties Missed': [penalties_missed],
                        'Penalties Saved': [penalties_saved],
                        'Assists': [assists],
                        'Player of the Match': [player_of_the_match],
                        'Season': [season],
                        'Matchday': [i]
                    })

                    df = pd.concat([df, new_row], ignore_index=True)



    return df

<br>

SCRAPING PHASE
<br>
<br>

In [ ]:
url = 'https://www.fantacalcio.it/'

try:
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.maximize_window()
    driver.get(url)

    # pop up closing
    pop_up_closer()

    # log-in
    log_in(user_id='cixi99', password='Abcdef1!')

    # pop up closing
    pop_up_closer()

    # fantacalcio grades
    megamenus = driver.find_elements(By.CLASS_NAME, 'megamenu')
    for megamenu in megamenus:
        if megamenu.text == 'Fantacalcio':
            megamenu.click()
            break
    driver.find_element(By.XPATH, "//a[@href='/voti-fantacalcio-serie-a']").click()

    # scrape seasons data
    final_df = scrape_data(season_list=['2023/24'])



finally:
    driver.quit()

# DATA MANIPULATION

In [ ]:
final_df.head(10)

,Name,Team,Role,Grade,FantaGrade,Goals Scored,Goals Conceded,Autogoals,Penalties Scored,Penalties Missed,Penalties Saved,Assists,Player of the Match,Season,Matchday
0,Musso,Atalanta,p,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
1,Zappacosta,Atalanta,d,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
2,Djimsiti,Atalanta,d,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,1
3,Kolasinac,Atalanta,d,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
4,Zortea,Atalanta,d,7.0,10.0,1,0,0,0,0,0,0,0,2023/24,1
5,Ruggeri,Atalanta,d,6.5,7.5,0,0,0,0,0,0,1,0,2023/24,1
6,Scalvini,Atalanta,d,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
7,De Roon,Atalanta,c,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1
8,Pasalic,Atalanta,c,5.5,5.5,0,0,0,0,0,0,0,0,2023/24,1
9,Adopo,Atalanta,c,6.5,7.5,0,0,0,0,0,0,1,0,2023/24,1


In [ ]:
# columns renaming
for column in final_df.columns:
    final_df = final_df.rename(columns={column : column.lower().replace(' ', '_')})

final_df.columns

Index(['name', 'team', 'role', 'grade', 'fantagrade', 'goals_scored',
       'goals_conceded', 'autogoals', 'penalties_scored', 'penalties_missed',
       'penalties_saved', 'assists', 'player_of_the_match', 'season',
       'matchday'],
      dtype='object')

In [ ]:
final_df = final_df.rename(columns={'name' : 'player_name',
                                    'team' : 'team_name'})
final_df.columns

Index(['player_name', 'team_name', 'role', 'grade', 'fantagrade',
       'goals_scored', 'goals_conceded', 'autogoals', 'penalties_scored',
       'penalties_missed', 'penalties_saved', 'assists', 'player_of_the_match',
       'season', 'matchday'],
      dtype='object')

In [ ]:
# role transformation
final_df['role'] = final_df['role'].replace({'p': 'Goalkeeper',
                                             'd': 'Defender',
                                             'c': 'Midfielder',
                                             'a': 'Attacker'})

final_df['role'].unique()

array(['Goalkeeper', 'Defender', 'Midfielder', 'Attacker'], dtype=object)

# SOME ANALYSIS

In [ ]:
# team name format understanding
final_df['team_name'].unique()

array(['Atalanta', 'Bologna', 'Cagliari', 'Empoli', 'Fiorentina',
       'Frosinone', 'Genoa', 'Inter', 'Juventus', 'Lazio', 'Lecce',
       'Milan', 'Monza', 'Napoli', 'Roma', 'Salernitana', 'Sassuolo',
       'Torino', 'Udinese', 'Verona'], dtype=object)

In [ ]:
# all players in the dataset (some players can occur with different teams)
final_df_unique_rows = final_df[['player_name', 'team_name', 'role']].drop_duplicates()
final_df_unique_rows

,name,team,role
0,Musso,Atalanta,Goalkeeper
1,Zappacosta,Atalanta,Defender
2,Djimsiti,Atalanta,Defender
3,Kolasinac,Atalanta,Defender
4,Zortea,Atalanta,Defender
...,...,...,...
11762,Nava,Milan,Goalkeeper
11765,Caldara,Milan,Defender
11792,Ferraris,Monza,Attacker
11886,Perilli,Verona,Goalkeeper


In [ ]:
# players that changed the team (with another italian team) during the same year
players_changed_team = (((final_df_unique_rows
                            .groupby('player_name'))
                            .filter(lambda x: x['team_name'].nunique() > 1)[['player_name', 'team_name']])
                            .sort_values(by='player_name'))

print(players_changed_team[:10],
     '\n----------------------------------\n',
     f'Number of players that changed team: {int(len(players_changed_team)/2)}')

           name         team
58     Baldanzi       Empoli
7116   Baldanzi         Roma
237     Belotti         Roma
6958    Belotti   Fiorentina
6677    Bohinen        Genoa
564     Bohinen  Salernitana
6039   Bonifazi    Frosinone
2863   Bonifazi      Bologna
7612  Cittadini        Genoa
5531  Cittadini        Monza 
----------------------------------
 Number of players that changed team: 26


In [ ]:
# players name format understanding
unique_names = [name for name in final_df_unique_rows['player_name']]

In [ ]:
# without spaces
category_1 = [name for name in unique_names if re.match(r'^[a-zA-Z]+$', name)]

print(category_1[:10], '\n-----------------------------------------\n', len(category_1))

['Musso', 'Zappacosta', 'Djimsiti', 'Kolasinac', 'Zortea', 'Ruggeri', 'Scalvini', 'Pasalic', 'Adopo', 'Koopmeiners'] 
-----------------------------------------
 498


In [ ]:
# with only letters and spaces
category_2 = [name for name in unique_names if re.search(r'\s', name)
                                              and re.match(r'^[a-zA-Z\s]+$', name)]

print(category_2[:10], '\n---------------------------------\n', len(category_2))

['De Roon', 'De Ketelaere', 'El Azzouzi', 'Van Hooijdonk', 'Di Pardo', 'Arthur Melo', 'De Vrij', 'Carlos Augusto', 'Alex Sandro', 'Iling Junior'] 
---------------------------------
 33


In [ ]:
# with only letter, spaces and points
category_3 = [name for name in unique_names if re.search(r'\.', name)
                                              and re.match(r'^[a-zA-Z\s\.]+$', name)]

print(category_3[:10], '\n---------------------------------\n', len(category_3))

['Ederson D.s.', 'Zapata D.', 'Moro N.', 'Sulemana I.', 'Pezzella Giu.', 'Henderson L.', 'Shpendi S.', 'Ranieri L.', 'Gonzalez N.', 'Beltran L.'] 
---------------------------------
 64


In [ ]:
# with more than one point
check_list_0 = [name for name in unique_names if len(re.findall(r'\.', name)) > 1
                                                and re.match(r'^[a-zA-Z\s\.]+$', name)]

# with points, but without spaces
check_list_1 = [name for name in unique_names if re.search(r'\.', name)
                                                and re.match(r'^[a-zA-Z\s\.]+$', name)
                                                and not re.search(r'\s', name)]

# without a point at the end of the string
check_list_2 = [name for name in unique_names if re.search(r'\.', name)
                                                and re.match(r'^[a-zA-Z\s\.]+$', name)
                                                and not re.search(r'\.$', name)]




# show the results
lists = [check_list_0, check_list_1, check_list_2]
dict_ = {
            0 : 'with more than one point',
            1 : 'with points, but without spaces',
            2 : 'without a point at the end of the string'}

for i in dict_.keys():
    if lists[i] != []:
        print(f'Words {dict_[i]}:\n', lists[i])
    else:
        print(f'No words {dict_[i]}')
    if i != 2:
        print('----------------------------------------------')

Words with more than one point:
 ['Ederson D.s.']
----------------------------------------------
No words with points, but without spaces
----------------------------------------------
No words without a point at the end of the string


In [ ]:
# the remaining ones
print(list(set(unique_names) -
           set(category_1) -
           set(category_2) -
           set(category_3)))

["N'Dicka", "Kouame'", "Lauriente'", "Traore' C.", "Mari'", 'Loftus-Cheek', "Montipo'", "D'Ambrosio", "Djalo'", "Toure' E.", "Ikone'", "Ake' M.", "Lucumi'", "Dodo'", 'Milinkovic-Savic V.', "Traore' Hj.", "Cisse' A.", "Soule'"]


In [ ]:
# with apostrophe
category_4 = [name for name in unique_names if re.search(r'\'', name)
                                              and re.match(r'^[a-zA-Z\']+$', name)]

print(category_4, '\n---------------------------------\n', len(category_4))

["Lucumi'", "Dodo'", "D'Ambrosio", "Mari'", "Lauriente'", "Montipo'", "Kouame'", "Soule'", "N'Dicka", "Ikone'", "Djalo'"] 
---------------------------------
 11


In [ ]:
# with more than one apostrophe
check_list_0 = [name for name in unique_names if len(re.findall(r'\'', name)) > 1
                                                and re.match(r'^[a-zA-Z\']+$', name)]

# with an apostrophe at the end of the name
check_list_1 = [name for name in unique_names if re.search(r'\'$', name)
                                                and re.match(r'^[a-zA-Z\']+$', name)]

# without an apostrophe at the end of the name
check_list_2 = [name for name in unique_names if re.search(r'\'', name)
                                                and re.match(r'^[a-zA-Z\']+$', name)
                                                and not re.search(r'\'$', name)]




# show the results
lists = [check_list_0, check_list_1, check_list_2]
dict_ = {
            0 : 'with more than one apostrophe',
            1 : 'with an apostrophe at the end of the name',
            2 : 'without an apostrophe at the end of the name'}

for i in dict_.keys():
    if lists[i] != []:
        print(f'Words {dict_[i]}:\n', lists[i])
    else:
        print(f'No words {dict_[i]}')
    if i != 2:
        print('----------------------------------------------')

No words with more than one apostrophe
----------------------------------------------
Words with an apostrophe at the end of the name:
 ["Lucumi'", "Dodo'", "Mari'", "Lauriente'", "Montipo'", "Kouame'", "Soule'", "Ikone'", "Djalo'"]
----------------------------------------------
Words without an apostrophe at the end of the name:
 ["D'Ambrosio", "N'Dicka"]


In [ ]:
# with dash
category_5 = [name for name in unique_names if re.search(r'\-', name)
                                              and re.match(r'^[a-zA-Z\-]+$', name)]

print(category_5, '\n---------------------------------\n', len(category_5))

['Loftus-Cheek'] 
---------------------------------
 1


In [ ]:
print(list(set(unique_names) -
           set(category_1) -
           set(category_2) -
           set(category_3) -
           set(category_4) -
           set(category_5)))

["Toure' E.", "Cisse' A.", "Ake' M.", "Traore' Hj.", 'Milinkovic-Savic V.', "Traore' C."]


# NAME STANDARDIZATION

In [ ]:
# shift the pointed part from the bottom of the string to the top of it
def rearrange_pointed_part(text):
    # pointed part
    match = re.search(r'(\s([a-zA-Z]+\.)+)$', text)
    if match:
        # initial space removal
        pointed_part = match.group().strip()
        # pointed part removal from the end of the string
        base_string = text[:match.start()]

        return f"{pointed_part} {base_string}"

    # if the pointed part doesn't exists, return th original string
    return text

In [ ]:
# funtion that add an apostrophe if the last letter has an accent
def add_apostrophe_to_final_accented_vowel(text):
    # set of vowels with grave or acute accents
    accented_vowels = {'à', 'á', 'è', 'é', 'ì', 'í', 'ò', 'ó', 'ù', 'ú'}
    words = text.split()

    # function to add an apostrophe after an accented vowel
    def process_word(word):
        if word[-1] in accented_vowels:
            return f'{word}\''
        return word

    # apply the function to each word
    processed_words = [process_word(word) for word in words]

    return ' '.join(processed_words)

In [ ]:
# funtion that remove the dicaritic characters
def diacritic_removal(text):
    # NFD decomposition
    normalized = unicodedata.normalize('NFD', text)

    return ''.join([char for char in normalized if not unicodedata.combining(char)])

In [ ]:
# function that substitute the 'special letters' with the simple ones
def normalize_special_letters(text):
    # Mappa di conversione delle lettere speciali e combinate
    special_letter_map = {
        'æ': 'ae', 'Æ': 'Ae',
        'œ': 'oe', 'Œ': 'Oe',
        'ß': 'ss',
        'Ĳ': 'IJ', 'ĳ': 'ij',
        'Đ': 'D', 'đ': 'd',
        'Ł': 'L', 'ł': 'l',
        'Ø': 'O', 'ø': 'o',
        'Ŋ': 'N', 'ŋ': 'n'
    }

    for letter, replacement in special_letter_map.items():
        text = text.replace(letter, replacement)

    # Rimuovi eventuali diacritici con NFD normalization
    normalized = unicodedata.normalize('NFD', text)
    text_without_special_letters = ''.join([char for char in normalized if not unicodedata.combining(char)])

    return text_without_special_letters

In [ ]:
# normalization pipeline
def name_normalization(row):
    text = row['player_name']
    text = rearrange_pointed_part(text)
    text = add_apostrophe_to_final_accented_vowel(text)
    text = diacritic_removal(text)
    text = normalize_special_letters(text)

    return text

In [ ]:
final_df['normalized_player_name'] = final_df.apply(name_normalization, axis=1)

final_df

,player_name,team_name,role,grade,fantagrade,goals_scored,goals_conceded,autogoals,penalties_scored,penalties_missed,penalties_saved,assists,player_of_the_match,season,matchday,normalized_player_name
0,Musso,Atalanta,Goalkeeper,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1,Musso
1,Zappacosta,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1,Zappacosta
2,Djimsiti,Atalanta,Defender,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,1,Djimsiti
3,Kolasinac,Atalanta,Defender,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,1,Kolasinac
4,Zortea,Atalanta,Defender,7.0,10.0,1,0,0,0,0,0,0,0,2023/24,1,Zortea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,Dani Silva,Verona,Midfielder,6.0,6.0,0,0,0,0,0,0,0,0,2023/24,38,Dani Silva
11898,Mitrovic S.,Verona,Midfielder,6.5,6.5,0,0,0,0,0,0,0,0,2023/24,38,S. Mitrovic
11899,Cisse' A.,Verona,Midfielder,55.0,55.0,0,0,0,0,0,0,0,0,2023/24,38,A. Cisse'
11900,Noslin,Verona,Attacker,7.5,11.5,1,0,0,0,0,0,1,0,2023/24,38,Noslin


In [ ]:
new_column_order = ['player_name', 'normalized_player_name', 'team_name', 'role', 'grade', 'fantagrade',
                    'goals_scored', 'goals_conceded', 'autogoals', 'penalties_scored', 'penalties_missed',
                    'penalties_saved', 'assists', 'player_of_the_match','season', 'matchday']

final_df = final_df[new_column_order]

In [ ]:
# save the dataframe
file_path = 'raw_data/df_players_grades.csv'
final_df.to_csv(file_path, index=False)